In [1]:
import pandas as pd

# 数据读取和数据分布

In [2]:
#数据读取，数据分布
data = pd.read_csv("./data/data_train.csv",header=None,sep='\t',encoding='gbk')

In [3]:
data.shape

(82025, 4)

In [4]:
data.columns = ['id','class','content','label'] #标签含义： 0：负面、1：中性、2：正面
data['content'] = data['content'].fillna('空') #填充空评论
data.head()

,id,class,content,label
0,1,食品餐饮,买这套系统本来是用来做我们公司的公众号第三方平台代运营的，没想到还有app，而且每个都很方便...,2
1,2,食品餐饮,烤鸭还是不错的，别的菜没什么特殊的,1
2,3,食品餐饮,使用说明看不懂！不会用，很多操作没详细标明！,0
3,4,食品餐饮,越来越不好了，菜品也少了，服务也不及时。,0
4,5,食品餐饮,是在是不知道该吃什么好、就来了,1


In [5]:
data['label'].value_counts() #积极评论65598，消极评论12240，中性评论4187

2    65598
0    12240
1     4187
Name: label, dtype: int64

In [6]:
data['class'].value_counts() 

医疗服务    20312
旅游住宿    18935
物流快递    14779
金融服务    14344
食品餐饮    13655
Name: class, dtype: int64

# 分词，去停用词

In [7]:
#分词
import jieba
data['word'] = data['content'].apply(lambda x:jieba.lcut(x))
data.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.926 seconds.
Prefix dict has been built succesfully.


,id,class,content,label,word
0,1,食品餐饮,买这套系统本来是用来做我们公司的公众号第三方平台代运营的，没想到还有app，而且每个都很方便...,2,"[买, 这套, 系统, 本来, 是, 用来, 做, 我们, 公司, 的, 公众, 号, 第三..."
1,2,食品餐饮,烤鸭还是不错的，别的菜没什么特殊的,1,"[烤鸭, 还是, 不错, 的, ，, 别的, 菜, 没什么, 特殊, 的]"
2,3,食品餐饮,使用说明看不懂！不会用，很多操作没详细标明！,0,"[使用, 说明, 看不懂, ！, 不会, 用, ，, 很多, 操作, 没, 详细, 标明, ！]"
3,4,食品餐饮,越来越不好了，菜品也少了，服务也不及时。,0,"[越来越, 不好, 了, ，, 菜品, 也, 少, 了, ，, 服务, 也, 不, 及时, 。]"
4,5,食品餐饮,是在是不知道该吃什么好、就来了,1,"[是, 在, 是, 不, 知道, 该, 吃, 什么, 好, 、, 就, 来, 了]"


In [8]:
#停用词表
stopwords = []
with open("./data/stopwords.txt",encoding='utf-8') as f:
    for line in f.readlines():
        stopwords.append(line.strip())
with open("./data/哈工大停用标点表.txt",encoding='utf-8') as f:
    for line in f.readlines():
        stopwords.append(line.strip())
with open("./data/哈工大停用词表扩展.txt",encoding='utf-8') as f:
    for line in f.readlines():
        stopwords.append(line.strip())

In [9]:
stopwords = set(stopwords)
print(len(stopwords))

1840


In [10]:
#去停用词
#arr 是一句话中分词后的词语构成的列表
# stopwords 是停用词构成的list
#返回 str,空格分隔
def filter_map(arr):
    res = ''
    for c in arr:
        if c and c not in stopwords and c != '\xa0'and c != '\n' and c != '\ufeff' and c != '\r':
            res+=c
            res+=' '
    return res.strip()

In [11]:
data['word'] = data['word'].apply(filter_map)
data.head()

,id,class,content,label,word
0,1,食品餐饮,买这套系统本来是用来做我们公司的公众号第三方平台代运营的，没想到还有app，而且每个都很方便...,2,买 这套 系统 本来 做 公司 公众 号 第三方 平台 代 运营 没想到 app 方便管理 ...
1,2,食品餐饮,烤鸭还是不错的，别的菜没什么特殊的,1,烤鸭 不错 菜 没什么
2,3,食品餐饮,使用说明看不懂！不会用，很多操作没详细标明！,0,看不懂 操作 详细 标明
3,4,食品餐饮,越来越不好了，菜品也少了，服务也不及时。,0,越来越 不好 菜品 少 服务
4,5,食品餐饮,是在是不知道该吃什么好、就来了,1,吃


# 划分 训练集，验证集，测试集，保存数据，便于后面使用

In [12]:
#打乱数据
from sklearn.utils import shuffle
data = shuffle(data,random_state=2019)
data.head()

,id,class,content,label,word
80669,80670,物流快递,我们单位跟雄图物流公司合作好些日子了，这家物流公司很负责，服务态度一流，我们老板也非常看好这...,2,单位 雄图 物流 公司 合作 好些 日子 这家 物流 公司 负责 服务态度 一流 老板 看好...
72575,72576,物流快递,赞一个，确实解决了不少的麻烦，工作人员也特别好，一直教到我会用，有问题打电话给他们也及时能帮...,2,赞 确实 解决 麻烦 工作人员 特别 教到 我会 打电话 帮 解决
77190,77191,物流快递,"雄图物流的保税区退运返修很专业，2小时通关,手续简单,还办理报关,商检,运输服务,提供返修场...",2,雄图 物流 保税区 退运 返修 专业 2 小时 通关 手续 简单 办理 报关 商检 运输 服...
58860,58861,医疗服务,里面的医疗设施很齐全，收费也标准，最关键是医生护士态度非常好，不错！,2,医疗 设施 齐全 收费 标准 关键 医生 护士 态度 不错
5584,5585,食品餐饮,点心、下午茶的出品还可以，只有点套餐内的披萨，真心让人失望，还不如回家自己烤,0,点心 下午茶 出品 套餐 披萨 真心 失望 回家 烤


In [13]:
#充分打乱
from sklearn.utils import shuffle
data = shuffle(data,random_state=0)
data.head()

,id,class,content,label,word
12521,12522,食品餐饮,技术经理很专业，培训也很到位，不愧是大品牌,2,技术 经理 专业 培训 到位 不愧 品牌
67726,67727,物流快递,快递太慢了，而且这个物流也不急时更新、这是最后一次用韵达快递，,0,快递 太慢 物流 急时 更新 这是 用韵 达 快递
58361,58362,医疗服务,上海复大医院帮助预约挂号的医生服务态度都很好，让我们着急的就诊的患者给我们安排的补插号，没有...,2,上海 复大 医院 预约 挂号 医生 服务态度 着急 就诊 患者 安排 补 插 号 耽误 看诊
70426,70427,物流快递,合作了一段时间，感觉无论时效还是专业度，都不错，过年前还有一票货，还是会选择他们。,2,合作 一段时间 感觉 时效 专业 度 不错 过年 前 一票 货 选择
23206,23207,旅游住宿,新疆很美，新疆很累，值得一游，景美。能把南疆包含最好,2,新疆 美 新疆 很累 值得 一游 景美 南疆 包含


In [27]:
#划分 训练集，验证集，测试集
import math
size = len(data)
pos_size = len(data[data['label']==2])
neu_size = len(data[data['label']==1])
neg_size = len(data[data['label']==0])

seg_pos1 = math.ceil(pos_size*0.8)
seg_pos2 = math.ceil(pos_size*0.9)

seg_neu1 = math.ceil(neu_size*0.8)
seg_neu2 = math.ceil(neu_size*0.9)

seg_neg1 = math.ceil(neg_size*0.8)
seg_neg2 = math.ceil(neg_size*0.9)

In [29]:
train = pd.concat([data[data['label']==2][:seg_pos1],data[data['label']==1][:seg_neu1],data[data['label']==0][:seg_neg1]],axis=0)
print(len(train))
train['label'].value_counts()

65621


2    52479
0     9792
1     3350
Name: label, dtype: int64

In [30]:
valid = pd.concat([data[data['label']==2][seg_pos1:seg_pos2],data[data['label']==1][seg_neu1:seg_neu2],data[data['label']==0][seg_neg1:seg_neg2]],axis=0)
print(len(valid))
valid['label'].value_counts()

8203


2    6560
0    1224
1     419
Name: label, dtype: int64

In [31]:
test= pd.concat([data[data['label']==2][seg_pos2:],data[data['label']==1][seg_neu2:],data[data['label']==0][seg_neg2:]],axis=0)
print(len(test))
test['label'].value_counts()

8201


2    6559
0    1224
1     418
Name: label, dtype: int64

In [38]:
#取到全部数据，且无重复取数据
assert len(train)+len(valid)+len(test)==size

In [37]:
#训练集:验证集:测试集= 8:1:1
print(len(train)/len(valid))
print(len(train)/len(test))
print(len(valid)/len(test))

7.999634280141412
8.001585172539935
1.0002438726984515


In [34]:
from sklearn.utils import shuffle
train = shuffle(train,random_state=2019)
train = shuffle(train,random_state=0)

valid = shuffle(valid,random_state=2019)
valid = shuffle(valid,random_state=0)

test = shuffle(test,random_state=2019)
test = shuffle(test,random_state=0)

In [42]:
#保存数据，便于后面使用
train.to_csv('./data/train.csv')
valid.to_csv('./data/valid.csv')
test.to_csv('./data/test.csv')

# 预训练词向量模型

In [43]:
train = pd.read_csv('./data/train.csv')
valid = pd.read_csv('./data/valid.csv')
test = pd.read_csv('./data/test.csv')
train['word'] =train['word'].fillna('')
valid['word'] =valid['word'].fillna('')
test['word'] =test['word'].fillna('')
texts = train['word'].tolist() + valid['word'].tolist() + test['word'].tolist()

In [45]:
texts = [t.strip().split() for t in texts]

In [47]:
len(texts)

82025

In [51]:
from gensim.models import Word2Vec
#sg=1,skip-gram模式
# size 词向量维度300
word_vec = Word2Vec(sentences =texts , sg=1, size=300, window=5,min_count=1, compute_loss=True,alpha=0.001)

In [52]:
print(word_vec.get_latest_training_loss())
print(word_vec)

22725236.0
Word2Vec(vocab=60422, size=300, alpha=0.001)


In [53]:
word_vec.wv.save_word2vec_format('./data/word_vec_300_skip.txt',binary = False)

In [55]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('./data/word_vec_300_skip.txt', binary=False)

# 检查词嵌入矩阵

In [56]:
import pickle
def save_file(obj,file):
    with open(file,'wb') as f:
        pickle.dump(obj, f)
def load_file(file):
    with open(file,'rb') as f:
        return pickle.load(f)

In [57]:
matrix = load_file('./data/emb_mtrix.pkl')

In [58]:
matrix

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.25070931e-02,  1.02960162e-01, -9.43669956e-03, ...,
        -2.24469870e-01,  2.86823250e-02,  9.79511738e-02],
       [ 4.09110263e-02,  1.51528835e-01, -1.31611496e-01, ...,
        -3.35956037e-01,  2.42104866e-02,  3.12138554e-02],
       ...,
       [ 7.30055675e-04,  9.25651984e-04,  9.01110063e-04, ...,
         7.76498870e-04, -8.54802551e-04, -8.95730220e-04],
       [-3.60838923e-04,  1.58436527e-03,  1.26347737e-03, ...,
        -2.12685904e-03, -8.65597220e-04,  1.95989385e-03],
       [ 4.31463763e-04, -1.31976369e-04,  4.24028869e-04, ...,
        -3.44230706e-04, -5.99334482e-04,  1.50126638e-03]])

In [59]:
matrix.shape

(59856, 300)